# Packages used


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import os.path
from tabulate import tabulate
from pprint import pprint
import nltk

In [ ]:
import random
import numpy
import scipy
import sklearn
import openpyxl

In [ ]:
#packages used
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
import pandas as pd
import pickle

# Sentiment Analyisis' training

The learning was done on a sample of 200 headlines, each with a "positive" or "negative" tag based on the news.

In [ ]:
#read the training file and clean it
df = pd.read_excel(r'C:\Users\Utente\OneDrive\Desktop\headlines_folder\training.xlsx')
df = df.replace("|", "").replace(",","").replace('"', '').replace(":","").replace("'","")
df = df.replace(".","").replace("A","").replace("Ã","").replace("©","").replace("Ã²","").replace("Ã²","")
df =df.replace("€™","")
df = df.dropna()

#divide the dataset in text and tag
x = df["text"] 
y = df["tag"]

#vectorize the text
vect = CountVectorizer(ngram_range=(1,1)) 
X = vect.fit_transform(x)

#split the data in training set(80%) and test set(20%)
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2)

#Bernoulli model
model = BernoulliNB()

#Model fitting
model.fit(X_train, y_train)

#results
model.predict(X_test)
print(model.predict(X_test)) 
print()
accuracy = accuracy_score(y_test, model.predict(X_test))
print(accuracy)

# Request newspapers URL

In [ ]:
#headlines tgcom
url1 = 'https://www.tgcom24.mediaset.it/'
response1 = requests.get(url1)
soup = bs(response1.text, 'html.parser')
headlines1 = soup.find('body').find_all('h2')
print()

#headline repubblica
url2= 'https://www.repubblica.it/'
response2 = requests.get(url2)
soup = bs(response2.text, 'html.parser')
headlines2 = soup.find('body').find_all('h2')

#headline fatto quotidiano
url3 = "https://www.ilfattoquotidiano.it/"
response3 = requests.get(url3)
soup = bs(response3.text, 'html.parser')
headlines3 = soup.find('body').find_all('h2')

#headline corriere
url4 = "https://www.corriere.it/"
response4 = requests.get(url4)
soup = bs(response4.text, 'html.parser')
headlines4 = soup.find('body').find_all("h4")


#headline ansa
url5 = "https://www.ansa.it/"
response5 = requests.get(url5)
soup = bs(response5.text, 'html.parser')
headlines5 = soup.find('body').find_all("h3")

total_headlines= [headlines1, headlines2, headlines3, headlines4, headlines5]



# Functions used

In [ ]:
#headlines scraper
def headlines_scraper():
    for x in total_headlines:   #loops that select each newspaper
        for y in x:             #loop that read the elements of each newspaper
            print(y.text.strip())
#headlines scraped in a list form
l_raw_text = []
for x in total_headlines:   #loops that read the list composed of sublist
    for y in x:             #loop that read the elements of the sublist(the headlines)
        l_raw_text.append(y.text.strip())
#percentage of negative news for sentiment analysis
def percentage_of_neg_news(lista):
    count = 0
    for i in lista:
        if i == 'n':
            count = count +1
    percentage = round((count / len(lista))*100 , 2)
    
    return percentage 

#mwnu options
def menu_options():
    print("------------------------------------------------------------------------------------")
    print('\033[1m' +"Select a number to perform the desired action:"+ '\033[0m')
    print("1 = Read today's news headlines on the most important italian newspapers")
    print("2 = Store today's headlines in the headlines_folder")
    print("3 = Read headlines from previous days from headlines_folder")
    print("4 = Most used words and word-combinations for a certain day")
    print("5 = Sentiment analysis evaluation for a certain day")
    print("6 = Terminate")
    print("------------------------------------------------------------------------------------")
    print()

# News Analyzer

**ATTENTION:**   For using the tools it's mandatory to create a folder named "headlines_folder" to store the data

In [ ]:
##### bella
print('\033[1m' +"NEWS ANALYZER"+'\033[0m')
print("Since ever evil sells and newspapers are prone to negative and catastrophic announcements.")
print("Fortunately, there is this tool to monitor this trend!")
print()

menu_options()

while True:
    CHOISE = input('\033[1m' +"Select an action: ")
    
    #Read all the today's news on the most important italian newspapers
    if(CHOISE == "1"): 
        print()
        print("Today's News: ")
        print()
        headlines_scraper()
        menu_options()
    #Store the headlines in the headlines_folder
    elif(CHOISE == "2"):
        save_path = r'C:\Users\Utente\OneDrive\Desktop\headlines_folder'
        file_name = input("What is the name of the file: ")
        #create the file and write on it today's headlines
        completeName = os.path.join(save_path, file_name + ".txt")
        file1 = open(completeName, "w", encoding="utf-8")
        #first row is text, it will be useful for the machine learnign model
        file1.write("text \n")
        #lowercase and write the headlines on the new file
        for i in l_raw_text:               
            lower_i = str(i.lower())  
            file1.write(lower_i + '\n') 
        file1.close()
        print()
        print("***file successfully saved*** ")
        print()
        menu_options()
    
    elif(CHOISE == "3"):
        #specify file path and file name
        save_path = r'C:\Users\Utente\OneDrive\Desktop\headlines_folder'
        file_name = input("What is the name of the file: ")
        #open the file and read headlines 
        print()
        f = open(save_path +"\\"+ file_name + ".txt", "r", encoding = 'utf-8') 
        row = f.readline()   # Read the first row of the file
        while row :          # while there are rows to read
            print(row)
            row = f.readline()
        f.close() # Mandatory
        print()
        menu_options()
   
    elif(CHOISE == "4"):
        #specify file path and file name
        save_path = r'C:\Users\Utente\OneDrive\Desktop\headlines_folder'
        #open the selected file and read the headlines
        selected_headlines = input("choose on what headlines data text perform the analysis: " )
        f = open(save_path +"\\"+ selected_headlines + ".txt", "r", encoding = 'utf-8')
        s_raw_text = f.read() 
        #clean the text from symbols
        s_raw_text_nosymb = s_raw_text.replace("|", "").replace(",","").replace('"', '').replace(":","").replace("'","").replace(".","") #pulisci i titoli dai simboli
        #transform the text into a list to take out the stopwords
        l_raw_text_nosymb: list[str] = nltk.word_tokenize(s_raw_text_nosymb) #transform s_raw_text_nosymb into a list
        
        stopwords = ["il","lo","la","i","gli","le","di","a","da", "in","con","su","per", "tra",\
             "fra","e","è","nel","dei","delle","degli","dello","del","dal","al"\
            ,'del','un','«','»','della','’','che','una','più','?','ecco','alla','(',')','si','l','dalla','come','ha','ma',\
            'ai','foto', 'nella','-', 'anche','sono','“', '”''sua', 'ci','ã¨', 'sulla', 'alle', 'de', 'ad', 'mi','sui',\
             "sul","prima",'sotto','sempre','fa','fino', 'ho','suo',"chi","non",'”','video',"mio",'dai',"così","uno",\
            "due","tre","ora","dopo","perché","%"]
       
        #loop for creating the clean_text list without stopwords
        clean_text = []
        for word in l_raw_text_nosymb :
            if word not in (stopwords):
                clean_text.append(word)
       
        #print in a table the 10 words with higest frequency
        
        fd = nltk.FreqDist(clean_text)
        print(tabulate(fd.most_common(10),headers = ["Most Common words", "frequency"],tablefmt="pretty"))
       
        #print in a table the 3 most common words' trio
        
        finder = nltk.collocations.TrigramCollocationFinder.from_words(clean_text)
        print(tabulate(finder.ngram_fd.most_common(3), headers=["Most Commons Words Trio ","frequency"],tablefmt="pretty"))
        print()
        menu_options()
        #aggiunta
        
    elif(CHOISE == "5"):
        save_path = r'C:\Users\Utente\OneDrive\Desktop\headlines_folder'
        #open the selected file and read the headlines

        selected_headlines = input("choose on which file perform the sentiment analysis: " )
        #if selected file exist in excel type than perform the sentiment analysis directly
        if os.path.exists(save_path +"\\"+ selected_headlines) == True:


            #quick text cleaning
            dfp = pd.read_excel(save_path +"\\"+ selected_headlines +'.xlsx')
            dfp.replace("|", "").replace(",","").replace('"', '').replace(":","").replace("'","").replace(".","").replace("A","").replace("Ã","").replace("©","")
            dfp = dfp.dropna()
            #perform the sentiment analysis
            x = dfp['text ']
            X = vect.transform(x)

            model_trained = pickle.load(open('model.pkl', 'rb'))
            vect = pickle.load(open('tfidf.pkl', 'rb'))
            X = vect.transform(x)

            model_trained.predict(X)

            print(model_trained.predict(X))
            #mshow negative news percentage

            print()
            print("The estimated percentage of negative news is: ", percentage)

        #if the file does not exist in excel format create it
        else:
            #create theexcel file
            read_file = pd.read_csv(save_path +"\\"+ selected_headlines + ".txt", on_bad_lines='skip')
            file_excel = read_file.to_excel(save_path +"\\"+ selected_headlines +'.xlsx', index = None, header=True)

            dfp = pd.read_excel(save_path +"\\"+ selected_headlines +'.xlsx')
            dfp.replace("|", "").replace(",","").replace('"', '').replace(":","").replace("'","").replace(".","").replace("A","").replace("Ã","").replace("©","")
            dfp = dfp.dropna()
            #perform the sentiment analysis
            x = dfp['text ']
            X = vect.transform(x)

            model_trained = pickle.load(open('model.pkl', 'rb'))
            vect = pickle.load(open('tfidf.pkl', 'rb'))
            X = vect.transform(x)

            result = model_trained.predict(X)

            print()
            print('\033[1m' +"The estimated negative news percentage is: "+'\033[0m', percentage_of_neg_news(result))
            print()
            menu_options()
    
    elif(CHOISE == "6"):
        break
        
        